In [1]:
import os
from Bio import PDB
from Bio.PDB.Superimposer import Superimposer
import pymol
import moviepy.editor as mpy

In [2]:
def align_structures(ref_structure, sample_structure):
    ref_atoms = []
    sample_atoms = []

    for ref_model in ref_structure:
        for ref_chain in ref_model:
            for ref_res in ref_chain:
                if ref_res.get_id()[0] == ' ':  # Check if it's not a hetero-atom
                    ref_atoms.append(ref_res['CA'])

    for sample_model in sample_structure:
        for sample_chain in sample_model:
            for sample_res in sample_chain:
                if sample_res.get_id()[0] == ' ':
                    sample_atoms.append(sample_res['CA'])

    super_imposer = Superimposer()
    super_imposer.set_atoms(ref_atoms, sample_atoms)
    super_imposer.apply(sample_structure.get_atoms())

In [3]:
from pathlib import Path

directory = Path('pdbs')
pattern = 'generated_0_timestep_'
matching_files = [f for f in directory.glob(f'{pattern}*.pdb')]

len(matching_files)

1000

In [4]:
filtered_files = []
for file in matching_files:
    try:
        # Extract the number part from the filename
        number = int(file.stem[len(pattern):])
        if 0 <= number <= 999:
            filtered_files.append((number, file.name))
    except ValueError:
        continue
len(filtered_files)

1000

In [5]:
filtered_files.sort()
sorted_filenames = [file[1] for file in filtered_files]
filtered_files[:10]

[(0, 'generated_0_timestep_0.pdb'),
 (1, 'generated_0_timestep_1.pdb'),
 (2, 'generated_0_timestep_2.pdb'),
 (3, 'generated_0_timestep_3.pdb'),
 (4, 'generated_0_timestep_4.pdb'),
 (5, 'generated_0_timestep_5.pdb'),
 (6, 'generated_0_timestep_6.pdb'),
 (7, 'generated_0_timestep_7.pdb'),
 (8, 'generated_0_timestep_8.pdb'),
 (9, 'generated_0_timestep_9.pdb')]

In [61]:
pdb_files = sorted_filenames[-60:]

In [62]:
parser = PDB.PDBParser()
ref_structure = parser.get_structure("reference", directory / pdb_files[0])

In [63]:
aligned_files = [directory / pdb_files[0]]

for i, pdb_file in enumerate(pdb_files[1:], start=1):
        sample_structure = parser.get_structure(f"sample_{i}", directory / pdb_file)
        align_structures(ref_structure, sample_structure)
        
        # Save aligned structure
        io = PDB.PDBIO()
        io.set_structure(sample_structure)
        aligned_pdb = directory / f"aligned_{i}.pdb"
        io.save(str(aligned_pdb))
        aligned_files.append(aligned_pdb)

In [64]:
    pymol.cmd.reinitialize()
    pymol.cmd.load(directory / pdb_files[0], "reference")
    pymol.cmd.zoom()
    pymol.cmd.center("reference")
    view = pymol.cmd.get_view()

In [65]:
    frames = []
    for pdb_file in aligned_files:

        pymol.cmd.reinitialize()
        pymol.cmd.load(pdb_file, "protein")
        pymol.cmd.set_view(view)  # Apply the same view for all structures
        pymol.cmd.rotate('y', 3 * len(frames))  # Cumulative rotation
        frame_name = f'frames/frame_{len(frames)}.png'
        pymol.cmd.png(frame_name)
        frames.append(frame_name)
        
        
        

In [66]:
output_file = "./pokus.mp4"

clip = mpy.ImageSequenceClip(frames, fps=4)
clip.write_videofile(output_file) if output_file.endswith('.mp4') else clip.write_gif(output_file)

Moviepy - Building video ./pokus.mp4.
Moviepy - Writing video ./pokus.mp4



Moviepy - Done !
Moviepy - video ready ./pokus.mp4


In [67]:
len(frames)

60